In [ ]:
# In this project I will create a deep learning classifier that takes in image data and classifies what type of galaxy it is. 

# the four possibilities of galaxies are..

# [1,0,0,0] - Galaxies with no identifying characteristics.
# [0,1,0,0] - Galaxies with rings.
# [0,0,1,0] - Galactic mergers.
# [0,0,0,1] - “Other,” Irregular celestial bodies.

In [ ]:
# Dataset from Galaxy Zoo
# https://www.zooniverse.org/projects/zookeeper/galaxy-zoo/,

In [4]:
## Codecademy functions to access the large galaxy data file

import requests
import io
import numpy as np
import os

#Loads data from url
def make_request(url):
    print("Requesting data from {}...".format(url))
    response = requests.get('https://content.codecademy.com/courses/deeplearning-with-tensorflow/'+url)
    response.raise_for_status()
    response_data = io.BytesIO(response.content)
    return response_data
    
#Loads galaxy data
def load_galaxy_data():
  
  #If cached file not found, loads data from url
  if not os.path.isfile('./cached_data.npz'):
     response_data = make_request(url='galaxydata.npz')

     with open("cached_data.npz","wb") as save_file:
      save_file.write(response_data.read())
 
  #Load data using NumPy
  data = np.load('cached_data.npz')

  print("Successfully loaded galaxy data!")
  
  return data["data"],data["labels"]

In [9]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from sklearn.model_selection import train_test_split



input_data, labels = load_galaxy_data()

Successfully loaded galaxy data!


In [8]:
print(input_data.shape)
print(labels.shape)

(1400, 128, 128, 3)
(1400, 4)


In [10]:
X_train, X_test, y_train, y_test = train_test_split(input_data, labels, test_size = .20,random_state  = 222, stratify=labels, shuffle = True)

In [13]:
generator = ImageDataGenerator(rescale = 1./255)

In [31]:
training = generator.flow(X_train,y_train,batch_size=5)
validation = generator.flow(X_test,y_test,batch_size=5)

In [32]:
from tensorflow.keras.layers import Conv2D

In [33]:
model = tf.keras.Sequential()

model.add(tf.keras.Input(shape =input_data.shape[1:4]))

model.add(Conv2D(8,3, strides = 2, activation = 'relu'))

model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2))

model.add(Conv2D(8,3, strides = 2, activation = 'relu'))

model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2))

model.add(tf.keras.layers.Flatten())

model.add(tf.keras.layers.Dense(16, activation= 'relu'))

model.add(tf.keras.layers.Dense(4, activation= 'softmax'))



In [59]:
model.compile(optimizer= tf.keras.optimizers.Adam(learning_rate =.001), loss= tf.keras.losses.CategoricalCrossentropy(), metrics=[tf.keras.metrics.Accuracy(),tf.keras.metrics.CategoricalAccuracy(), tf.keras.metrics.AUC()])

In [60]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 63, 63, 8)         224       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 31, 31, 8)        0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 15, 15, 8)         584       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 7, 7, 8)          0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 392)               0         
                                                                 
 dense_3 (Dense)             (None, 16)               

In [61]:
history = model.fit(training, steps_per_epoch = len(training)/5,epochs  = 12, validation_data = validation,validation_steps = len(validation)/5)

Epoch 1/12
44/44 [==============================] - 1s 15ms/step - loss: 0.9259 - accuracy: 0.0000e+00 - categorical_accuracy: 0.6400 - auc_7: 0.8491 - val_loss: 1.0257 - val_accuracy: 0.0000e+00 - val_categorical_accuracy: 0.5167 - val_auc_7: 0.8091
Epoch 2/12
44/44 [==============================] - 0s 8ms/step - loss: 1.0001 - accuracy: 0.0000e+00 - categorical_accuracy: 0.5689 - auc_7: 0.8236 - val_loss: 1.0725 - val_accuracy: 0.0000e+00 - val_categorical_accuracy: 0.6167 - val_auc_7: 0.8001
Epoch 3/12
44/44 [==============================] - 0s 8ms/step - loss: 0.9945 - accuracy: 0.0000e+00 - categorical_accuracy: 0.5867 - auc_7: 0.8254 - val_loss: 0.9790 - val_accuracy: 0.0000e+00 - val_categorical_accuracy: 0.6333 - val_auc_7: 0.8337
Epoch 4/12
44/44 [==============================] - 0s 8ms/step - loss: 1.0201 - accuracy: 0.0000e+00 - categorical_accuracy: 0.5644 - auc_7: 0.8168 - val_loss: 0.9683 - val_accuracy: 0.0000e+00 - val_categorical_accuracy: 0.6333 - val_auc_7: 0.8392

In [62]:
results = model.evaluate(X_test, y_test)

9/9 [==============================] - 0s 11ms/step - loss: 108.1165 - accuracy: 0.6821 - categorical_accuracy: 0.5214 - auc_7: 0.6896


In [63]:
print(results)

[108.11646270751953, 0.6821428537368774, 0.5214285850524902, 0.6895768642425537]


In [64]:
model.metrics_names

['loss', 'accuracy', 'categorical_accuracy', 'auc_7']

In [ ]:
# My model has an accuracy of .68. This means that my model correctly classifies deep-space galaxies 68 percent of the time.
# This is fairly good considering that randomly guessing would result in a accuracy of 25 percent because of the four categories.